<a href="https://colab.research.google.com/github/santiagonajera/optimizacionInventrios/blob/main/Optimizacion_Invetarios.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import pandas as pd
import requests
from io import BytesIO
import numpy as np
from scipy import stats
pd.set_option('display.float_format', lambda x: '%.1f' % x)  # Configurar formato global
pd.set_option('display.max_columns', None)  # Mostrar todas las columnas
pd.set_option('display.width', None)  # Usar todo el ancho disponible
pd.set_option('display.max_rows', None)  # Mostrar todas las filas

# URL del archivo Excel
url = 'https://github.com/santiagonajera/optimizacionInventrios/raw/refs/heads/main/EjemploClases-Eco.xlsx'

# Descargar el archivo Excel
response = requests.get(url)
excel_file = BytesIO(response.content)

# Leer las hojas 'Forecast', 'Precios-Costos', 'Historico', 'LeadTime-Dias'
df_forecast = pd.read_excel(excel_file, sheet_name='Forecast')
df_precios = pd.read_excel(excel_file, sheet_name='Precios-Costos')
df_historico = pd.read_excel(excel_file, sheet_name='Historico')
df_lead_time = pd.read_excel(excel_file, sheet_name='LeadTime-Dias')

# Calcular las ventas totales por item
df_forecast['Venta Total'] = df_forecast.iloc[:, 1:].sum(axis=1).round(1)

# Fusionar los DataFrames para agregar los precios
df = pd.merge(df_forecast, df_precios[['ITEM', 'Precio']], on='ITEM')

# Calcular las ventas totales en dólares
df['Venta Total Dolares'] = (df['Venta Total'] * df['Precio']).round(1)

# Ordenar de mayor a menor según la venta total en dólares
df = df.sort_values(by='Venta Total Dolares', ascending=False)

# Calcular el porcentaje acumulado de ventas
df['Venta Acumulada'] = df['Venta Total Dolares'].cumsum().round(1)
df['Porcentaje Acumulado'] = (df['Venta Acumulada'] / df['Venta Total Dolares'].sum() * 100).round(1)

# Clasificación ABC
df['Clasificación ABC'] = pd.cut(df['Porcentaje Acumulado'],
                                bins=[0, 60, 80, 100],
                                labels=['A', 'B', 'C'],
                                include_lowest=True)

# Calcular el coeficiente de variación para cada item
df_historico = df_historico.iloc[:, -18:]  # Tomar los últimos 18 datos
df_coef_var = df_historico.apply(lambda x: (np.std(x) / np.mean(x)).round(1), axis=1)

# Clasificación XYZ usando límites fijos en lugar de percentiles
def clasificar_xyz(cv):
    if cv <= 0.5:
        return 'X'
    elif cv <= 1.0:
        return 'Y'
    else:
        return 'Z'

df['Clasificación XYZ'] = df_coef_var.apply(clasificar_xyz)

# Unir clasificaciones ABC y XYZ
df['Clasificación ABC-XYZ'] = df['Clasificación ABC'].astype(str) + '' + df['Clasificación XYZ'].astype(str)

# Calcular promedio y desviación estándar para cada item
df_promedio = df_historico.mean(axis=1).round(1)
df_desv_est = df_historico.std(axis=1).round(1)

# Asignar nivel de servicio (95%)
nivel_servicio = 0.95
z_alfa = stats.norm.ppf(nivel_servicio).round(1)

# Calcular lead time
lead_time = (df_lead_time.iloc[:, 1] + 0.5).round(1)
lead_time_div = (lead_time / 30).round(1)
f_lt = np.sqrt(lead_time_div).round(1)

# Calcular stock de seguridad
stock_seguridad = (z_alfa * f_lt * df_desv_est).round(1)

# Calcular stock de seguridad en días
stock_seg_dias = ((stock_seguridad / df_promedio) * 30).round(1)

# Crear DataFrame de resultados con formato mejorado
resultados = pd.DataFrame({
    'ITEM': df['ITEM'],
    'Clasif ABC-XYZ': df['Clasificación ABC-XYZ'],
    'Promedio': df_promedio,
    'Desv. Est.': df_desv_est,
    'F.NSer': z_alfa,
    'F.Lt': f_lt,
    'Stock Seg.': stock_seguridad,
    'Stock Seg. Días': stock_seg_dias
})

# Si existe una columna de descripción en df_precios, la agregamos
if 'DESCRIPCION' in df_precios.columns:
    resultados.insert(1, 'Descripción', df_precios.set_index('ITEM')['DESCRIPCION'])

# Crear una tabla con estilo
tabla_estilizada = resultados.style\
    .set_properties(**{'text-align': 'center'})\
    .format(precision=1, thousands=",")\
    .set_table_styles([
        {'selector': 'th', 'props': [('text-align', 'center'), ('background-color', '#f2f2f2')]},
        {'selector': 'td', 'props': [('text-align', 'center')]}
    ])

# Mostrar la tabla resultante
print("\nTabla de Resultados:")
print(resultados.to_string(index=False))

# También podemos exportar a Excel para mejor visualización
resultados.to_excel('resultados_inventario.xlsx', index=False, float_format='%.1f')
print("\nLos resultados también han sido exportados a 'resultados_inventario.xlsx'")


Tabla de Resultados:
   ITEM Clasif ABC-XYZ  Promedio  Desv. Est.  F.NSer  F.Lt  Stock Seg.  Stock Seg. Días
 ITEM 1             BX   60294.0     12474.5     1.6   0.3      5987.8              3.0
 ITEM 2             AX  106479.3     22342.0     1.6   0.7     25023.0              7.1
 ITEM 3             AX   82444.7     18584.8     1.6   0.3      8920.7              3.2
 ITEM 4             AX   93980.7     19791.5     1.6   0.8     25333.1              8.1
 ITEM 5             BX   59085.6     13650.5     1.6   0.6     13104.5              6.7
 ITEM 6             CX   52718.0     11854.4     1.6   0.8     15173.6              8.6
 ITEM 7             AX   95447.7     20900.2     1.6   0.4     13376.1              4.2
 ITEM 8             CX   52001.7     10492.0     1.6   0.5      8393.6              4.8
 ITEM 9             CX   50709.0     11327.4     1.6   0.7     12686.7              7.5
ITEM 10             BX   50302.4     10560.0     1.6   0.5      8448.0              5.0

Los resul